In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import projectlib
from projectlib.projlib import download_dataset
import keras
from keras import preprocessing
from keras import layers,models,datasets
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import pathlib
import tensorflow_hub as hub 
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import matplotlib.image as pltimg
from numpy import expand_dims

In [4]:
model = VGG16()
images = tf.keras.utils.image_dataset_from_directory("/Users/kousthubhveturi/Downloads/datasetmri/Training")
images.class_names
imageclasses = ['glioma', 'meningioma', 'notumor', 'pituitary']

for image in images:
    print(image)



img_t = "/Users/kousthubhveturi/Downloads/datasetmri/Testing"
ds_t = tf.keras.preprocessing.image_dataset_from_directory(img_t)

2022-08-15 13:47:24.285254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 5712 files belonging to 4 classes.
(<tf.Tensor: shape=(32, 256, 256, 3), dtype=float32, numpy=
array([[[[0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         ...,
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ]],

        [[0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         ...,
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ]],

        [[0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         ...,
         [1.  , 1.  , 1.  ],
         [1.  , 1.  , 1.  ],
         [1.  , 1.  , 1.  ]],

        ...,

        [[0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         ...,
         [1.  , 1.  , 1.  ],
         [1.  , 1.  , 1.  ],
         [1.  , 1.  , 1.  ]],

        [[0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  ],
         ...,
         [0.  , 0.  , 0.

In [8]:
# the Visual Geometry Group from Oxford
# 16 and 19 layers pretrained (paper is written in 2015)
# keras has pretrained for both, we will use 16 here
#model.summary()
# if want model to be interpretable, wouldn't be using deep network. deep network learns millions of parameters, say, i want this decision to be made for millions of middle reasons
# 
print(type(model))


<class 'keras.engine.functional.Functional'>
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_con

In [10]:
from numpy import save
import pathlib
import os
from pathlib import Path
count_num = 0
os.environ["MRI_DATA_PATH"] = "/Users/kousthubhveturi/Downloads/datasetmri/"
def generate_vgg_features():
    '''
    before calling function, define following vars:
    environment variable to access directory for training data
    call environment variable: "MRI_DATA_PATH", points to the dataset
        
    '''
    model = VGG16()
    kept_layers = model.layers[:-1]
    model = keras.Sequential(kept_layers)
    source = Path(os.environ["MRI_DATA_PATH"],"training/") 
    images = tf.keras.utils.image_dataset_from_directory(source)
    dest = Path("artifacts/vgg_features/training/")
    dest.mkdir(parents=True,exist_ok=True)

    num = 0
    for image in images:
        a = tf.image.resize(image[0],[224,224])
        pred = model.predict(a)
        
        output_path = str(num)
        save(os.path.join(dest,output_path),pred)
        print("saved")
        num = num+1
    dest_t = Path("artifacts/vgg_features/testing/")
    dest_t.mkdir(parents=True,exist_ok=True)
    num = 0
    for image in ds_t:
        a = tf.image.resize(image[0],[224,224])
        pred = model.predict(a)
        output_path = str(num)
        save(os.path.join(dest_t,output_path),pred)
        print("saved")
        num = num+1
    count_num = num
generate_vgg_features()
# if having rgb valeus represented as 32 bit integeres, something isnt straight

Found 5712 files belonging to 4 classes.
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 4s 4s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 6s 6s/step
saved
1/1 [==============================] - 6s 6s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 4s 4s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 5s 5s/step
saved
1/1 [==============================] - 4s 4s/step
saved
1/1 [==

In [11]:
from sklearn.neighbors import NearestNeighbors 
import pickle
def train_nearest_neighbor_classifier():
      arr = [None] * 178
      for i in range(178):
            training_npyfile = np.load(("/Users/kousthubhveturi/Desktop/Research - Summer 2022/artifacts/vgg_features/training/"+str(i)+".npy"))
            arr[i] = training_npyfile
      sample, x, y = np.shape(arr)
      shapedarray = np.reshape(arr,(sample,x*y))
      nnc = NearestNeighbors(n_neighbors=4)
      nnc.fit(shapedarray)
      dest = Path("artifacts/models/")
      dest.mkdir(parents=True,exist_ok=True)
      pickle.dump(nnc,open("artifacts/models/nearest_neighbor_classifier.pkl","wb"))
      print("saved model")

train_nearest_neighbor_classifier()

saved model


In [27]:
array_file72 = np.load("/Users/kousthubhveturi/Desktop/Research - Summer 2022/artifacts/vgg_features/training/72.npy")
arr = array_file72
print(arr.shape)
 

(32, 1000)


In [25]:
def test_nearest_neighbor_classifier():
 
      with open("artifacts/models/nearest_neighbor_classifier.pkl","rb") as f:
            predict_model = pickle.load(f)
      arr = [None] * 40
      for i in range(len(arr)):
            testing_npfile = np.load(("/Users/kousthubhveturi/Desktop/Research - Summer 2022/artifacts/vgg_features/testing/"+str(i)+".npy"))
            arr[i] = testing_npfile
      sample, x, y = np.shape(arr)
      shapedarray = np.reshape(arr,(sample,x*y))
      predicted_results = [None] * count_num
      for i in arr:
            predicted_results[i] = predict_model.fit(arr[i])   
      test_y = np.concatenate(ds_t,1)
      predict_model.score(predicted_results,test_y)
test_nearest_neighbor_classifier() 
      

TypeError: only integer scalar arrays can be converted to a scalar index